# Chapter 7 합성곱 신경망(CNN)

## 7.1 전체 구조

CNN에서 등장하는 새로운 layer
- 합성곱 계층(convolutional layer)
- 풀링 계층(pooling layer)

현재까지 봤던 신경망을 완전연결(fully-connected)라고 하며 Affine 계층으로 구현했다. Affine 계층 뒤에는 활성화 함수로 ReLU 혹은 Sigmoid를 사용했다.

CNN의 구조는 새로운 합성곱 계층(Conv)과 풀링 계층(pooling)이 추가된다. Conv-ReLU-(Pooling)의 흐름이다. 출력층과 가까운 층의 경우에는 Affine-ReLU, 출력층은 Affine-Softmax 조합을 그대로 사용한다.

<img src="https://t1.daumcdn.net/cfile/tistory/2409463658F46CAD1F">
<center><small>▲ 완전연결 계층(위), CNN(아래)</small></center>

## 7.2 합성곱 계층

- 패딩(padding)
- 스트라이드(stride)
- 입체적인 데이터 흐름

### 7.2.1 완전연결 계층의 문제점

데이터의 형상이 무시된다.
- 완전연결 계층의 입력은 평탄화 필요
- 본래 다차원인 데이터의 경우 다차원에서 특별한 정보가 담겨있을 가능성이 있다.
- 이미지의 경우
    - 가로, 세로, 색상의 3차원 데이터
    - 공간적 정보
    
반면, 합성곱 계층은 형상을 유지한다.
- 입력 데이터가 형상 그대로 들어온다.
- 다음 계층으로 전달될 때도 그대로 전달
- 다차원의 형상을 가진 데이터를 올바르게 이해할 수 있다.
- 특징 맵(feature map): 합성곱 계층의 입출력 데이터
    - 입력 특징 맵(input feature map)
    - 출력 특징 맵(output feature map)

### 7.2.2 합성곱 연산

합성곱 연산 = 필터 연산


합성곱 연산 예제
<img src="https://t1.daumcdn.net/cfile/tistory/2764173558F475B42C">

데이터 설명
- 입력 데이터: (4,4)의 높이와 너비를 가진 형상
- 필터: (3,3)의 높이와 너비를 가진 형상
    - 커널이라고도 한다.
- 출력: (2,2)의 놆이와 너비를 가진 형상

연산 과정
1. 필터의 **윈도우(window)**를 일정 간격으로 이동하면서 입력 데이터에 적용
2. 단일 곱셈-누산(fused multiply-add, FMA): 대응하는 원소기리 곱한 후 모두 더함
3. 결과를 출력의 해당 장소에 저장
4. 모든 장소에서 수행

가중치와 편향
- 가중치: 필터의 매개변수
- 편향: 필터를 적용한 후 데이터에 더해진다. 항상 하나(1X1)만 존재

### 7.2.3 패딩

패딩(padding)
- 입력 데이터 주변을 특정 값으로 채우는 것
- 예를 들어 0으로

패딩의 예

<img src="https://t1.daumcdn.net/cfile/tistory/2527FA3758F4785B13">

- 입력데이터: 4X4
- 패딩 후: 6X6
- 3X3 필터 적용 후: 4X4

패딩을 하는 이유
- 출력 크기를 조정하는 목적
- 합성곱 신경망에서 그냥 필터를 적용하면 계속해서 크기가 줄어든다. 신경망이 깊어지면 어느 순간 크기가 1이 되어버린다. 이는 더이상 합성곱을 할 수 없는 상태이기 때문에 문제가 된다.
- 풀링을 하면 출력 크기를 유지시켜 줄 수 있어서, 입력 데이터의 공간적 크기를 고정해서 다음 층으로 넘겨줄 수 있다.

### 7.2.4 스트라이드

스트라이드
- 필터를 적용하는 위치 간격
- 스트라이드를 키우면 출력 크기가 작아진다.

<img src="https://2.bp.blogspot.com/-vtZW1-cBQGg/WYJrUnBjRiI/AAAAAAAALNY/GhTnu5QDi3M4NHB_FiyOJAjy58mTkzlYwCK4BGAYYCw/s320/o9.PNG">
<center><small>▲ 스트라이드가 2인 합성곱 신경망</small></center>

패딩, 스트라이드, 출력 크기 계산
- 입력 크기: $(H, W)$
- 필터 크기: $(FH, FW)$
- 출력 크기: $(OH, OW)$
- 패딩: $P$
- 스트라이드: $S$

$$OH = \frac {H + 2P - FH} S + 1$$

$$OW = \frac {W + 2P - FW} S + 1$$

주의
- 계산 결과가 정수로 나누어 떨어져야 한다.

### 7.2.5 3차원 데이터의 합성곱 연산

3차원 데이터의 합성곱 연산 예

<img src="https://t1.daumcdn.net/cfile/tistory/99C185405BC97F4D1E">

- 입력 데이터의 채널 수 = 필터의 채널 수
- 필터의 크기는 원하는 크기로(모든 채널의 필터 크기는 같아야 함)

### 7.2.6 블록으로 생각하기

<img src="https://t1.daumcdn.net/cfile/tistory/998CE7355BC97F632E">

- 데이터와 필터의 형상: (채널, 높이, 너비)
- 출력: 채널이 1개인 특징 맵

<img src="https://t1.daumcdn.net/cfile/tistory/99CDF2395BC97F7C2D">

- 필터를 여러 개 사용하면 출력의 채널 수도 늘어남
- 이 출력을 다음 층으로 넘겨준다.
- 필터의 가중치 데이터는 4차원: (출력 채널 수, 입력 채널 수, 높이, 너비)
- 편향: 채널당 하나의 값

### 7.2.7 배치 처리

<img src="https://t1.daumcdn.net/cfile/tistory/99E4C84E5C4D31B728">

- 4차원으로 데이터 저장: (데이터 수, 채널 수, 높이, 너비)
- 가장 앞쪽에 배치용 차원을 추가
- 각 흐름마다 N번의 합성곱 연산을 수행
- 배치 처리의 효과는 완전연결 신경망과 동일

## 7.3 풀링 계층

풀링 계층
- 가로, 세로 방향의 공간을 줄이는 연산

풀링의 예

<img src="https://t1.daumcdn.net/cfile/tistory/993A5B465C4D32D32C">

- 최대 풀링(max pooling)
    - 대상 영역 내에서 최대값을 구하는 연산
- 평균 풀링(average pooling)
    - 대상 영역의 평균을 계산
- 이미지 인식 분야에서는 최대 풀링을 사용
- 윈도우 크기와 스트라이드는 동일한 값으로 하는 것이 일반적
    - 위의 예: 윈도우 2X2, 스트라이드 2

### 7.3.1 풀링 계층의 특징

학습해야 할 매개변수가 없다.

채널 수가 변하지 않는다.
- 채널마다 독립적으로 계산하기 때문

입력의 변화에 영향을 적게 받는다(강건하다).
- 입력 데이터가 조금 변해도 풀링 결과는 잘 변하지 않는다.

## 7.4 합성곱/풀링 계층 구현하기

### 7.4.1 4차원 배열

CNN에서 흐르는 데이터는 4차원이다.

In [2]:
import numpy as np

In [3]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [4]:
# 첫번째 데이터에 접근
x[0].shape

(1, 28, 28)

In [5]:
# 두번째 데이터에 접근
x[1].shape

(1, 28, 28)

In [ ]:
# 첫번째 데이터의 첫 채널에 접근
x[0,0]